# DADOS QUANTITATIVOS DISCRETOS

In [3]:
%matplotlib notebook

from pandas.tools.plotting import scatter_matrix
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

tabela = 'dataset/quantitativos/velocidade_internet.csv'
coluna = 'velocidade'

dataset = pd.read_csv(tabela)
dataset = dataset[coluna]
#data_ordenado = data.sort_values()
#data_ordenado

# Usando pandas para criar a series de numeros
#data = dataset['velocidade']
ordenados = sorted(dataset)
data = pd.Series(ordenados)
#data

# Montando a tabela de distribuição de frequência

#math.ceil arrendonda arredonda para o número inteiro mais próximo em direção ao infinito positivo
tamanho = len(data)
valor_minimo = min(data)
qnt_classe = math.ceil(math.sqrt(tamanho))

amplitude = data.max() - data.min()
intervalo = round(amplitude / qnt_classe,2)

# Intervalos
inicial = valor_minimo
s = valor_minimo

#Meu Proprio intervalo para recorte dos dados
intervalos = []
for i in range(qnt_classe + 1):
    intervalos.append(inicial)
    inicial+= intervalo

#print(intervalos)

df = pd.DataFrame(data,columns=['Classes'])
#df

factor = pd.cut(df['Classes'],intervalos)
print(factor)
frequencia = pd.value_counts(factor)

#Agrupando dados para contagem
frequencia = df.groupby(factor)['Classes'].agg(['count'])
#print(frequencia)

# Salvando tabela inicial
frequencia.to_csv('dataset_gerados/tabela_teste2.csv')
dataset_df = pd.read_csv('dataset_gerados/tabela_teste2.csv')

# Calculando a frequencia acumulada
frequencia = dataset_df['count']
freq_acumulada = []
fa = 0
for i in range(0,len(frequencia)):
    fa += frequencia[i]
    freq_acumulada.append(fa);    

dataset_df.rename(columns={'count':'F'}, inplace=True)
dataset_df['FA'] = freq_acumulada
#dataset_df

# Calcular a frequencia relativa
total = sum(frequencia)
dataset_df['FR %'] = dataset_df['F'] / total
#dataset_df

#Calcular a frequencia relativa acumulada
frequencia = dataset_df['FR %']
freq_relativa_acumulada = []
fra = 0
for i in range(0,len(frequencia)):
    fra += frequencia[i]
    freq_relativa_acumulada.append(fra);    

dataset_df['FRA %'] = freq_relativa_acumulada


# Ponto médio
xi = []
s = 0
for i in range(qnt_classe):
    xi.append(intervalos[i])

dataset_df['XI'] = xi + (intervalo / 2)
#dataset_df

# Frequencia acumulada do ponto médio
frequencia = dataset_df['XI']
freq_ponto_medio = []
fxi = 0
for i in range(0,len(frequencia)):
    fxi += frequencia[i]
    freq_ponto_medio.append(fxi);    

dataset_df['F.XI'] = freq_ponto_medio
#dataset_df

# Ponto Médio ao quadrado
dataset_df['XI2'] = dataset_df['XI'] ** 2 
#dataset_df

# Frequencia do Ponto Médio ao quadrado
frequencia = dataset_df['XI2']
freq_ponto_medio_quadrado = []
fxi2 = 0
for i in range(0,len(frequencia)):
    fxi2 += frequencia[i]
    freq_ponto_medio_quadrado.append(fxi2); 
dataset_df['F.XI2'] = freq_ponto_medio_quadrado

# XI.LI
dataset_df['XI.LI'] = dataset_df['XI'] * dataset_df['F']

#new_df = pd.DataFrame(dataset_df)

# Salvando tabela inicial
#new_df.to_csv('dataset_gerados/tabela_teste.csv',index=False)
#new_df

0              NaN
1     (0.63, 1.44]
2     (0.63, 1.44]
3     (0.63, 1.44]
4     (0.63, 1.44]
5     (1.44, 2.25]
6     (1.44, 2.25]
7     (1.44, 2.25]
8     (1.44, 2.25]
9     (2.25, 3.06]
10    (2.25, 3.06]
11    (2.25, 3.06]
12    (2.25, 3.06]
13    (2.25, 3.06]
14    (2.25, 3.06]
15    (3.06, 3.87]
16    (3.06, 3.87]
17    (3.06, 3.87]
18    (3.06, 3.87]
19    (3.87, 4.68]
20    (3.87, 4.68]
21    (3.87, 4.68]
22    (4.68, 5.49]
23    (4.68, 5.49]
24     (5.49, 6.3]
25     (5.49, 6.3]
26     (5.49, 6.3]
27     (5.49, 6.3]
28     (5.49, 6.3]
29     (5.49, 6.3]
30     (5.49, 6.3]
31     (5.49, 6.3]
32     (5.49, 6.3]
33     (5.49, 6.3]
34     (5.49, 6.3]
35     (5.49, 6.3]
36     (5.49, 6.3]
37     (5.49, 6.3]
38     (5.49, 6.3]
39     (5.49, 6.3]
Name: Classes, dtype: category
Categories (7, interval[float64]): [(0.63, 1.44] < (1.44, 2.25] < (2.25, 3.06] < (3.06, 3.87] < (3.87, 4.68] < (4.68, 5.49] < (5.49, 6.3]]


# Calculos da Tabela

In [6]:
df = pd.read_csv('dataset_gerados/tabela_teste.csv')
#df

# Media
df['F'].sum()
#df['XI.LI']


# Moda

# Mediana

# Variancia

# Quartil

# Percentil

# Desvio Padrão

# Coeficiente de variação

39

### Usando os recursos da linguagem

In [2]:
# amplitude, variancia, desvio padrão, coeficiente de variação

print("Média: %.2f " % data.mean())
#print("Moda: %s " % t.mode())

print("Mediana: %.2f " % data.median())
print("Quartil: %.2f " % data.quantile(q=0.25))

percentil = np.percentile(data,16)
print("Percentil: %.2f " % percentil)

soma = sum(data)
maximo = max(data)
minimo = min(data)
amplitude = maximo - minimo

print("Amplitude: %2.f " % amplitude)
print("Variancia: %f" % data.var())
print("Desvio Padrão: %f " % data.std())
print("Coeficiente de Variação: ")

# MATPLOTLIB: Histograma (univariado)
#vel_internet = pd.Series(data) # Transformanda os dados em um serie
# vel_internet.hist()
# plt.show()


Média: 3.39 
Mediana: 3.31 
Quartil: 3.25 
Percentil: 3.23 
Amplitude:  1 
Variancia: 0.037140
Desvio Padrão: 0.192717 
Coeficiente de Variação: 


### Media
A média indica o valor onde estão concetrados os dados de um conjuto de valores, representando um valor
significativo para o mesmo.

In [2]:
print("Média: ", data.mean())

Média:  3.9965


### Moda
A moda é o valor que mais se repete dentro de um conjunto

In [17]:
#print("Moda: %s " % vel_internet.mode())

Moda: 0    2.32
1    2.90
dtype: float64 


### Mediana
A media é o valor central de um conjunto, ou seja, ela separa a metade superior da metade inferior de uma distribuição

In [18]:
#qnt_filhos_ord = sorted(qnt_filhos)
#centro = tam/2 
#mediana = qnt_filhos_ord[int(centro)] # centro = 8
#print("Mediana: %d " % mediana)

#print("Mediana: %d " % vel_internet.median())

Mediana: 4 


### Quartil

In [19]:
#print("Quartil: %d " % vel_internet.quantile(q=0.25)) #Primeiro quartil

Quartil: 2 


### Percentil

In [21]:
percentil = np.percentile(vel_internet,16)
#print("Percentil: %2.f " % percentil)

Percentil:  2 


## Medidas de dispersão
São medidas que indicam o quão espalhados estão os dados, ou sua variação. São elas: Amplitude, variância, desvio padrão e desvio absoluto

In [3]:
qnt_acidentes = pd.Series([3,5,0,2,3,5,9,0,2,1,1,0,1,8,5,1,1,7,2,0])

dataset1 = [4,9,2,8,0,6,1,8,7,0,
           4,3,8,9,8,6,2,9,9,3,
           3,3,8,5,0,9,7,3,4,0,
           7,0
          ]

dataset2 = [5,4,3,6,2,2,7]

data1 = pd.Series(dataset1)
data2 = pd.Series(dataset2)

#print(sorted(qnt_acidentes))

# Descrição rapida dos dados
# count = Quantidade de dados
# mean = Média
# sdt = Desvio Padrão
# min = Valor Minimo
# 25% = 1° Quartil
# 50% = 1° Quartil
# 75% = 1° Quartil
# max = Valor máximo
#print(qnt_acidentes.describe())
#percentil = np.percentile(qnt_acidentes, 73)
# print("Percentil 73: ", percentil ) # VER ISSO

### Amplitude
#### A = maximo - minimo
A amplitude é a diferença entre o maior e menor valor de um conjunto de dados.

In [12]:
minimo = qnt_acidentes.min()
maximo = qnt_acidentes.max()

print("Máximo: %d Mínimo: %d" % (maximo, minimo) )
print("Amplitude: %d " % (maximo - minimo))

Máximo: 9 Mínimo: 0
Amplitude: 9 


### Variancia 
#### (S²) - Amostral
#### (O²) - Populacional
A variância indica quanto os dados de um conjunto estão afastados de seu valor esperado.

In [9]:
import math

# Variancia Populacional
def varianciaPopulacional(data):
    tamanho = len(data)
    media = data.mean()
    soma = 0
    
    for valor in data:
        soma += math.pow( (valor - media ), 2)
     
    return soma / float(tamanho)


# Variancia Amostral
def varianciaAmostral(data):
    tamanho = len(data)
    somaDados = sum(data)

    soma1 = 0
    soma2 = (math.pow(somaDados, 2)) / tamanho
    
    for valor in dataset:
        soma1 += math.pow((valor),2)
        
    return (soma1 - soma2) / float((tamanho - 1))
    
#print("Variancia Populacional: ", varianciaPopulacional(data1))
print("Variancia Amostral: ", varianciaAmostral(data1)) # 4,75


774.7948
Variancia Amostral:  3.484982307692305


### Desvio Padrão
É a raiz quadrada da variância

In [83]:
def desvioPadrao(data):
    return math.sqrt(data)

print("Desvio padrão Amostral: ", desvioPadrao(varianciaPopulacional(data1)))
print("Desvio padrão Populacional: ", desvioPadrao(varianciaAmostral(data2)))

Desvio padrão Amostral:  3.163358332136908
Desvio padrão Populacional:  1.9518001458970666


### Coeficiente de Variação

In [85]:
print("Populacional: ", (data1.std()/data1.mean())*100)
print("Amostral: ", (data2.std()/data2.mean())*100)

Populacional:  66.35303802690034
Amostral:  47.11241731475677


### Mediana da tabela

m/2 = 40/2 = 20º e 21º

Essas posições estão respectivamente na 4º e 5º classes

#### hi = Li - li 
-> hi - 4 - 3 = 1

h- = Amplitude da classe i
Li = Limite superior na classe i
li = Limite inferior da classe i


hi/li = 1/5 = 0,2

#### PMQ - FANT 
-> 20 - 15 = 5

Pmq = Posição da mediana
Fant = Frequência acumulada anterior

### Para a posição 21º
MQ = lit(0,2) * 5 
-> 3 + 1 = 4
    
#### MQ = li + hi *(PMQ - Fant) / hi

### Para a posição 21º
MQ = 4 + 1 * (21 -20) / 3
MQ = 4 + 1 * 1 / 3
MQ = 4 + 1 / 3

= 4,33


### Quartil - Divide em quatro partes iguais


m/4 = 40/4 = 10º posição(q1)

esta na classe 3º posição

#### 1º Quartil = q1 = li + hi(Pq1 - Fant)/li
2 + 1*(10-7)/8

2+3/8 = 2,38

#### 3º Quartil = q3 = li + hi(Pq3 - Fant)/li
5 + 1 * (30 -23)/13
5 + 7/13
= 5,54


### Percentil
p72
#### m / 100
40/100 = 0,4 * 72 = 28,8 POSIÇÃO
está na 6º classe da tabela

p72 = li + hi(Pq - Fant)/fi

= 5 + 1*(28,8 - 23)/13
= 5+ 5,8/13 = 5,45